In [2]:
import os
import pandas as pd
import numpy as np
import json
import string
import re
from tqdm import tqdm
import pandas as pd
import glob
import pickle

Merge csv

In [14]:
all_files = glob.glob(os.path.join("./result_minie/a*r.csv"))
combined_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_csv.to_csv("./result_minie/abstract.csv", index=False)

Remove

In [2]:
df = pd.read_csv("./result_minie/abstract.csv")
sorted_df = df.sort_values("subject")

In [3]:
sorted_df.head()

,subject,relation,object
5757675,#,were elevated in,deceased patients
5757674,#,were,elevated
2762704,#,is,clinicaltrials.gov
379823,# 7131,be compared to,# QUANT_O_1 in usual care group with QUANT_O_2...
50381,# QUANT_S_1,is,QUANT_O_1


In [5]:
def isalnum(text):
    return text.replace(" ", "").replace("-", "").replace("_", "").isalnum()

In [6]:
def isNaN(num):
    return num != num

In [22]:
def same(row1, row2):
    return row1.subject == row2.subject and row1.relation == row2.relation and row1.object == row2.object

In [23]:
last_row = sorted_df.iloc[1]
result = []
for i in tqdm(range(len(sorted_df.index))):
    row = sorted_df.iloc[i]
    # remove NaN
    if not isNaN(row.subject) and not isNaN(row.object):
        # remove special symbol
        if isalnum(row.subject) and isalnum(row.object):
            # remove the entities longer than 3 words
            if len(row.subject.split()) <4 and len(row.object.split()) <4:
                # different from the previous line
                if not same(row, last_row):
                    last_row = row
                    result.append(row)

100%|█████████████████████████████████████████████████████████████████████| 6210457/6210457 [08:33<00:00, 12087.56it/s]


In [ ]:
pickle.dump(result, open('./result_minie/abstract_cleaned.sav', 'wb'))

SAV to CSV

In [2]:
result = pickle.load(open('./result_minie/abstract_cleaned.sav', 'rb'))

In [4]:
results = pd.DataFrame(result)

In [5]:
results.to_csv('minie-post-re.csv', index=False)

Create entity

In [54]:
df = pd.read_csv("./minie-post-re.csv")

In [55]:
set_1 = set(df['subject'])
set_2 = set(df['object'])
set_entityies = list(set_1) + list(set_2 - set_1)
results = pd.DataFrame(list(set_entityies))
results.columns = ["entity_name"]
sorted_df = results.sort_values("entity_name")
sorted_df['entity_id'] = range(1, len(sorted_df) + 1)

In [57]:
order = ['entity_id','entity_name']
sorted_df = sorted_df[order]

In [58]:
sorted_df

,entity_id,entity_name
326835,1,-RSB-
339021,2,-RSB- apelin-13
447713,3,-RSB- concentrations
900007,4,-RSB- existence
677031,5,-RSB- famous herb
...,...,...
82011,1019611,ﬁrst
606279,1019612,ﬁrst paper
537493,1019613,𝒪 memory
1012366,1019614,𝛼


In [60]:
sorted_df.to_csv('minie-entity-neo4j.csv', index=False)

Then use /assets/database.sh